In [68]:
import pandas as pd

import requests
import time

In [70]:
CURRENCIES = {
    'Bitcoin' : 'BTC', 
    'Ethereum' : 'ETH', 
    'Litecoin' : 'LTC',
    'AdEx' : 'ADX', 
    'Substratum' : 'SUB',
    'Ripple' : 'XRP',
    'NEO' : 'NEO',
    'IOTA' : 'IOT',
    'Monero' : 'XMR',
             }
TARGET_CURRENCY = 'USD'

In [71]:
PRICE_HISTORY_BASE_URL = 'https://min-api.cryptocompare.com/data/histoday'
REQUEST_PARAMS = '?fsym={source_currency}&tsym={target_currency}&aggregate=1&e=CCCAGG&allData=1'
TEMPLATE_REQUEST_URL = PRICE_HISTORY_BASE_URL + REQUEST_PARAMS

In [72]:
# Obtain historical data from the beginning of time for each currency
currency_dataframe = {}
for currency, symbol in CURRENCIES.items():
    request_url = TEMPLATE_REQUEST_URL.format(source_currency = symbol, target_currency = TARGET_CURRENCY)
    print('Fetching historical market data for {} ({} -> {}) ....'.format(currency, symbol, TARGET_CURRENCY))
    print('Request URL :', request_url)
    json_response = requests.get(url = request_url).json()
    if json_response['Response'] == 'Error':
        print('Failed to fetch historical market data for {} ({} -> {}) !, Error Message : {}'
              .format(currency, symbol, TARGET_CURRENCY, json_response['Message']))
    else:
        # print(json_response)
        df = pd.DataFrame(json_response['Data'])
        # convert time (unix timestamp) to date
        df['date'] = pd.to_datetime(df['time'], unit = 's')
        # drop time column
        df.drop('time', axis = 1, inplace = True)
        # set date column as index
        df.set_index('date', inplace = True)
        
        # volumefrom : trading volume of source currency traded (ex: BTC)
        # volumeto : trading volume of target currency traded (ex: USD)
        # average price of 1 unit of the source currency in the target currency = volumeto / volumefrom
        df.rename(columns = {'open': 'market_open', 'close': 'market_close',
                             'low': 'market_low', 'high': 'market_high',
                             'volumefrom': 'trading_volume_source_currency', 
                             'volumeto': 'trading_volume_target_currency'}, inplace = True)
        currency_dataframe[currency] = df
        print('Successfully fetched historical market data for {} !'.format(currency, symbol))
    # sleep for 1s to avoid hitting max rate limit per second
    time.sleep(1)
        

Fetching historical market data for Bitcoin (BTC -> USD) ....
Request URL : https://min-api.cryptocompare.com/data/histoday?fsym=BTC&tsym=USD&aggregate=1&e=CCCAGG&allData=1
Successfully fetched historical market data for Bitcoin !
Fetching historical market data for Ethereum (ETH -> USD) ....
Request URL : https://min-api.cryptocompare.com/data/histoday?fsym=ETH&tsym=USD&aggregate=1&e=CCCAGG&allData=1
Successfully fetched historical market data for Ethereum !
Fetching historical market data for Litecoin (LTC -> USD) ....
Request URL : https://min-api.cryptocompare.com/data/histoday?fsym=LTC&tsym=USD&aggregate=1&e=CCCAGG&allData=1
Successfully fetched historical market data for Litecoin !
Fetching historical market data for AdEx (ADX -> USD) ....
Request URL : https://min-api.cryptocompare.com/data/histoday?fsym=ADX&tsym=USD&aggregate=1&e=CCCAGG&allData=1
Successfully fetched historical market data for AdEx !
Fetching historical market data for Substratum (SUB -> USD) ....
Request URL :

In [73]:
df = currency_dataframe['Bitcoin']

In [74]:
df.head()

,market_close,market_high,market_low,market_open,trading_volume_source_currency,trading_volume_target_currency
date,,,,,,
2010-07-17,0.04951,0.04951,0.04951,0.04951,20.00,0.9902
2010-07-18,0.08584,0.08585,0.05941,0.04951,75.01,5.0900
2010-07-19,0.08080,0.09307,0.07723,0.08584,574.00,49.6600
2010-07-20,0.07474,0.08181,0.07426,0.08080,262.00,20.5900
2010-07-21,0.07921,0.07921,0.06634,0.07474,575.00,42.2600


In [75]:
df.describe()

,market_close,market_high,market_low,market_open,trading_volume_source_currency,trading_volume_target_currency
count,2822.000000,2822.000000,2822.000000,2822.000000,2822.000000,2.822000e+03
mean,1070.716964,1111.836518,1017.611038,1068.270447,55680.937119,1.215253e+08
std,2621.147044,2740.407404,2462.713891,2618.932416,56294.426398,4.131862e+08
min,0.049510,0.049510,0.010000,0.049510,0.000000,0.000000e+00
25%,10.870000,11.122500,10.532500,10.862500,18021.547500,3.849676e+05
50%,254.925000,261.685000,247.130000,254.680000,40217.285000,6.112285e+06
75%,624.397500,634.025000,610.400000,624.012500,73819.057500,2.626149e+07
max,19345.490000,19870.620000,18750.910000,19346.600000,572349.320000,6.245732e+09


In [61]:
df.shape

(2822, 6)